In [1]:
import pandas as pd
import numpy as np
import re

from elder_care.config import BLD
from elder_care.config import SRC

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
def table(df_col):
    return pd.crosstab(df_col, columns='Count')["Count"]

In [4]:
def process_gv_imputations(wave, args):
    module = "gv_imputations"
    module_file = SRC / f"data/sharew{wave}/sharew{wave}_rel8-0-0_{module}.dta"
    data = pd.read_stata(module_file, convert_categoricals=False)

    # Filter the data based on the "country" column
    data = data[data["country"] == 12]

    # # Select columns 'mergeid' and the specified args
    # selected_columns = ["mergeid"] + args
    # data = data[selected_columns]

    # Select columns 'mergeid' and the specified args (create missing columns with NaN)
    # Filter existing columns
    selected_columns = ["mergeid"] + [col for col in args if col in data.columns]
    columns = ["mergeid"] + args

    # Create missing columns and fill with NaN
    for col in args:
        if col not in selected_columns:
            data[col] = np.nan

    # data = data[selected_columns]
    data = data[columns]
    
    # Replace negative values with NaN using NumPy
    # data[args] = np.where(data[args] >= 0, data[args], np.nan)

    # Group the data by 'mergeid'
    grouped_data = data.groupby("mergeid")

    # Create a dictionary to store the aggregation method for each column
    aggregation_methods = {}
    for column in args:
        dtype = data[column].dtype
        if pd.api.types.is_integer_dtype(dtype):
            aggregation_methods[column] = "median"
        elif pd.api.types.is_float_dtype(dtype):
            aggregation_methods[column] = "mean"

    # Replace negative values with NaN using NumPy
    # this should not change the meaning except for cases where all 5 entries are missing
    # check ?!
    data[args] = np.where(data[args] >= 0, data[args], np.nan)
    #data[args] = np.where(data[args] >= 0, data[args], 0)
            
    # Apply aggregation methods and store the results in a new DataFrame
    aggregated_data = grouped_data.agg(aggregation_methods).reset_index()
    
    #if "age_p" in args:
    #    # note that single people also have partner_alive = 0
    #    aggregated_data['partner_alive'] = np.where(aggregated_data['age_p'] > 0, 1, 0)
    
    aggregated_data["wave"] = wave

    return aggregated_data

In [5]:
# Sample data
data = {
    'ID': [1, 1, 1, 1, 1, 2, 2, 2, 2, 2],
    'Values': [1, 2, np.nan, 1,  np.nan, np.nan, 1, 3, 5, np.nan]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Group by 'ID'
grouped_data = df.groupby('ID')

# Define aggregation methods
aggregation_methods = {
    'Values': "median"
}

# Apply aggregation methods and store the results in a new DataFrame
aggregated_data = grouped_data.agg(aggregation_methods).reset_index()

aggregated_data

,ID,Values
0,1,1.0
1,2,3.0


In [6]:
wave = 2
module = "gv_imputations"
module_file = SRC / f"data/sharew{wave}/sharew{wave}_rel8-0-0_{module}.dta"
data = pd.read_stata(module_file, convert_categoricals=False)
data = data[data["country"] == 12]

In [7]:
args = [
    "gender",
    "age",  # Age of respondent (based on interview year)
    "age_p",  # Age of partner (based on interview year)
    "mstat", # Marital status
    "single",
    "couple",
    "partner",
    "nursinghome",
    # "perho",
    "ydip",
    "yind",
    "ypen1",  # Annual old age, early retirement pensions, survivor and war pension
    "ypen2",  # Annual private occupational pensions
    "ypen5",  # Annual payment from social assistance
    "yreg1",  # Other regular payments from private pensions
    "yreg2",  # Other regular payment from private transfer
    "thinc",  # Total household net income - version A
    "thinc2",  # Total household net income - version B
    "hnetw",  # Household net worth (hnfass + hrass)
    "yedu",
    "yedu_p",
    "isced",
    "sphus",  # Self-perceived health – US scale
    "nchild",  # Number of children
    "gali",  # Limitation with activities: PH005
    "chronic",  # Number of chronic diseases: PH006
    "adl",  # Limitations with activities of daily living: PH049_1
    "iadl",  # Limitations with instrumental activities of daily living: PH049_2
    "eurod",  # EURO depression scale: MH002-MH017 (MH031)
    "cjs",  # Current job situation: EP005
    "pwork",  # Did any paid work: EP002
    "empstat",  # Employee or self-employed: EP009; 2- 8
    "rhfo",  # Received help from others (how many): SP002, SP005, SP007
    "ghto",  # Given help to others (how many): SP008, SP011, SP013
    "ghih",  # Given help in the household (how many): SP0181 2 4 5 6 7 (R) 8
    "rhih",  # Received help in the household (how many): SP0201 2 4 5 6 7 (R) 8
    "otrf",  # Owner, tenant or rent free: HO0021 2 4 5 6 7 (R) 8
]
#args = ["mstat"]

# thinc: (sum of ydip, ypen1, ypen2, ypen3, ypen4, ypen5,
# ypen6, yreg1, yreg2, yind, ysrent, yaohm, ybabsmf)

# hrass: Household real assets
# (home*perho/100+vbus*sbus/100+car+ ores – mort)
# hgfass: Household gross financial assets (sum of
# bacc, bsmf and slti)
# hnfass: Household net financial assets (hgfass – liab)

# Select columns 'mergeid' and the specified args (create missing columns with NaN)
# Filter existing columns
selected_columns = ["mergeid"] + [
    col for col in args if col in data.columns
] 
columns = ["mergeid"] + args

# Create missing columns and fill with NaN
for col in args:
    if col not in selected_columns:
        data[col] = np.nan 

# data = data[selected_columns]
data = data[columns]


# Group the data by 'mergeid'
grouped_data = data.groupby("mergeid")

In [8]:
result_df = process_gv_imputations(8, args)
# result_df["single"].isna().sum()
result_df["couple"].isna().sum()

table(result_df["ghto"])

ghto
0.0    1853
1.0     701
2.0     240
3.0      84
Name: Count, dtype: int64

In [9]:
table(result_df["ghih"])

ghih
0.0    2064
1.0     196
2.0       3
Name: Count, dtype: int64

In [10]:
table(data["age_p"])

age_p
-99    2590
 25       5
 33       5
 35       5
 36       5
 37       5
 38      10
 39      15
 40       5
 41      10
 42      10
 43      45
 44      40
 45      25
 46      20
 47      30
 48      65
 49      95
 50     120
 51     225
 52     265
 53     330
 54     395
 55     406
 56     455
 57     460
 58     431
 59     377
 60     305
 61     376
 62     260
 63     465
 64     440
 65     305
 66     515
 67     460
 68     355
 69     455
 70     385
 71     382
 72     325
 73     242
 74     160
 75     145
 76     160
 77     155
 78     160
 79     125
 80     106
 81     120
 82      90
 83      65
 84      55
 85      30
 86      30
 87       5
 88      10
 89       5
Name: Count, dtype: int64

In [11]:
(data["age_p"] < 40).sum() / 5, data.shape[0] / 5

(528.0, 2628.0)

In [12]:
def test_negative_age_couple_condition(data):
    # Check that if 'age_p' is negative or missing, 'couple' is 0
    condition = ((data['age_p'] < 0) | data['age_p'].isna()) == (data['couple'] == 0)
    assert condition.all()

In [13]:
test_negative_age_couple_condition(data)

In [14]:
data[(data['age_p'] < 0) | (data['age_p'].isna()) & (data['couple'] == 0) & (data['single'] == 1)].shape

(2590, 37)

In [15]:
data[(data['age_p'] < 0) | (data['age_p'].isna()) & (data['couple'] == 0)].shape

(2590, 37)

In [16]:
data[(data['age_p'] < 0) | (data['age_p'].isna()) & (data['single'] == 0)].shape

(2590, 37)

In [17]:
aggregation_methods = {}
for column in args:
    dtype = data[column].dtype
    if pd.api.types.is_integer_dtype(dtype):
        aggregation_methods[column] = 'median'
    elif pd.api.types.is_float_dtype(dtype):
        aggregation_methods[column] = 'mean'

In [18]:
aggregation_methods

{'gender': 'median',
 'age': 'median',
 'age_p': 'median',
 'mstat': 'median',
 'single': 'median',
 'couple': 'median',
 'partner': 'median',
 'nursinghome': 'median',
 'ydip': 'mean',
 'yind': 'mean',
 'ypen1': 'mean',
 'ypen2': 'mean',
 'ypen5': 'mean',
 'yreg1': 'mean',
 'yreg2': 'mean',
 'thinc': 'mean',
 'thinc2': 'mean',
 'hnetw': 'mean',
 'yedu': 'mean',
 'yedu_p': 'mean',
 'isced': 'median',
 'sphus': 'median',
 'nchild': 'median',
 'gali': 'median',
 'chronic': 'median',
 'adl': 'median',
 'iadl': 'median',
 'eurod': 'median',
 'cjs': 'median',
 'pwork': 'median',
 'empstat': 'median',
 'rhfo': 'median',
 'ghto': 'median',
 'ghih': 'median',
 'rhih': 'median',
 'otrf': 'median'}

In [19]:
aggregated_data = grouped_data.agg(aggregation_methods).reset_index()
aggregated_data

,mergeid,gender,age,age_p,mstat,single,couple,partner,nursinghome,ydip,yind,ypen1,ypen2,ypen5,yreg1,yreg2,thinc,thinc2,hnetw,yedu,yedu_p,isced,sphus,nchild,gali,chronic,adl,iadl,eurod,cjs,pwork,empstat,rhfo,ghto,ghih,rhih,otrf
0,DE-000132-01,2.0,53.0,-99.0,5.0,1.0,0.0,-99.0,0.0,20000.000000,0.0,0.0,0.0,0.0,0.0,0.0,21850.000000,22896.000000,113310.178834,17.0,-99.0,5.0,1.0,2.0,1.0,0.0,0.0,0.0,2.0,2.0,-99.0,1.0,2.0,1.0,0.0,0.0,1.0
1,DE-001381-01,2.0,53.0,61.0,5.0,0.0,1.0,1.0,0.0,1200.000000,0.0,0.0,0.0,0.0,0.0,0.0,8995.740918,12000.000000,32779.784972,13.0,11.0,5.0,5.0,2.0,0.0,1.0,0.0,0.0,5.0,3.0,1.0,1.0,3.0,2.0,0.0,-99.0,3.0
2,DE-001381-02,1.0,61.0,53.0,5.0,0.0,1.0,2.0,0.0,900.000000,0.0,0.0,0.0,0.0,0.0,0.0,8995.740918,12000.000000,32779.784972,11.0,13.0,3.0,5.0,2.0,1.0,5.0,0.0,0.0,6.0,3.0,0.0,-99.0,3.0,1.0,0.0,0.0,3.0
3,DE-001748-01,1.0,50.0,43.0,1.0,0.0,1.0,1.0,0.0,18591.740766,0.0,0.0,0.0,0.0,0.0,0.0,22254.731445,62846.383338,39821.574948,9.0,13.0,2.0,4.0,1.0,1.0,1.0,0.0,0.0,1.0,2.0,-99.0,1.0,0.0,1.0,0.0,0.0,3.0
4,DE-001748-02,2.0,43.0,50.0,1.0,0.0,1.0,2.0,0.0,1100.000000,0.0,0.0,0.0,0.0,0.0,0.0,22254.731445,62846.383338,39821.574948,13.0,9.0,3.0,2.0,1.0,0.0,0.0,0.0,0.0,2.0,2.0,-99.0,1.0,0.0,0.0,0.0,-99.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2623,DE-998490-02,2.0,50.0,59.0,2.0,0.0,1.0,2.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,15900.000000,14400.000000,10000.000000,12.0,12.0,3.0,4.0,2.0,1.0,1.0,0.0,0.0,6.0,3.0,0.0,-99.0,3.0,1.0,0.0,0.0,4.0
2624,DE-999375-01,1.0,70.0,68.0,1.0,0.0,1.0,1.0,0.0,0.000000,0.0,15600.0,1200.0,0.0,0.0,0.0,21690.134375,21600.000000,138869.948142,10.0,18.0,3.0,5.0,3.0,1.0,6.0,0.0,1.0,6.0,1.0,0.0,-99.0,0.0,1.0,0.0,-99.0,1.0
2625,DE-999375-02,2.0,68.0,70.0,1.0,0.0,1.0,2.0,0.0,0.000000,0.0,4800.0,0.0,0.0,0.0,0.0,21690.134375,21600.000000,138869.948142,18.0,10.0,3.0,4.0,3.0,1.0,7.0,3.0,2.0,6.0,5.0,0.0,-99.0,0.0,0.0,0.0,0.0,1.0
2626,DE-999404-01,1.0,60.0,58.0,4.0,0.0,1.0,1.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,4800.000000,14400.000000,69058.099347,10.0,12.0,3.0,3.0,4.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,-99.0,0.0,0.0,0.0,0.0,1.0


In [20]:
(aggregated_data['single'] == 'couple').sum()

0

In [21]:
((aggregated_data['single'] == 0) & (aggregated_data['couple'] == 0)).sum()

0

In [22]:
result_df = process_gv_imputations(8, args)
# result_df["single"].isna().sum()
result_df["couple"].isna().sum()

0

In [23]:
pd.crosstab(data['mstat'], data['couple'])

couple,0,1
mstat,,
1,60,9905
2,5,61
3,95,46
4,454,166
5,577,202
6,1399,170


In [24]:
result_df['partner_alive'] = np.where(result_df['age_p'] > 0, 1, 0)

In [25]:
# 1, 2, 3
result_df[
    ((result_df["mstat"] == 1) | (result_df["mstat"] == 2) | (result_df["mstat"] == 3))
    & ((result_df["age_p"] < 0) | (result_df["age_p"].isna()))
]

,mergeid,gender,age,age_p,mstat,single,couple,partner,nursinghome,ydip,yind,ypen1,ypen2,ypen5,yreg1,yreg2,thinc,thinc2,hnetw,yedu,yedu_p,isced,sphus,nchild,gali,chronic,adl,iadl,eurod,cjs,pwork,empstat,rhfo,ghto,ghih,rhih,otrf,wave,partner_alive
67,DE-022865-01,2.0,72.0,NaN,1.0,1.0,0.0,NaN,0.0,0.0,0.0,27600.000000,0.0,0.000000,0.0,0.0,28100.000000,27600.0,412000.000000,25.0,NaN,5.0,2.0,1.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,NaN,1.0,0.0,NaN,NaN,1.0,8,0
113,DE-043035-01,2.0,65.0,NaN,3.0,1.0,0.0,NaN,0.0,16000.0,0.0,0.000000,0.0,0.000000,0.0,0.0,16188.263867,14400.0,93169.164179,15.0,NaN,4.0,5.0,2.0,1.0,5.0,0.0,0.0,6.0,2.0,NaN,1.0,2.0,1.0,NaN,NaN,3.0,8,0
132,DE-051099-01,1.0,76.0,NaN,1.0,1.0,0.0,NaN,0.0,0.0,0.0,11400.000000,0.0,0.000000,0.0,0.0,11600.000000,17400.0,3689.752249,15.0,NaN,5.0,3.0,0.0,1.0,5.0,0.0,0.0,1.0,1.0,0.0,NaN,0.0,1.0,NaN,NaN,3.0,8,0
220,DE-081577-01,1.0,75.0,NaN,3.0,1.0,0.0,NaN,0.0,0.0,0.0,14400.000000,0.0,0.000000,0.0,0.0,14400.000000,14400.0,3513.691221,11.0,NaN,3.0,4.0,3.0,1.0,3.0,0.0,0.0,3.0,1.0,0.0,NaN,0.0,0.0,NaN,NaN,3.0,8,0
248,DE-092185-01,2.0,77.0,NaN,2.0,1.0,0.0,NaN,0.0,0.0,0.0,15600.000000,0.0,0.000000,0.0,0.0,15610.000000,15600.0,163000.000000,11.0,NaN,3.0,3.0,1.0,1.0,2.0,0.0,0.0,0.0,1.0,0.0,NaN,0.0,0.0,NaN,NaN,1.0,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2739,DE-947557-02,2.0,83.0,NaN,1.0,1.0,0.0,NaN,0.0,0.0,0.0,7440.000000,0.0,624.820659,0.0,0.0,31285.452734,30000.0,16874.458822,12.0,NaN,3.0,5.0,0.0,1.0,1.0,4.0,5.0,6.0,1.0,0.0,NaN,2.0,0.0,0.0,0.0,3.0,8,0
2757,DE-953610-01,2.0,77.0,NaN,1.0,1.0,0.0,NaN,0.0,0.0,0.0,18000.000000,0.0,0.000000,0.0,0.0,18000.000000,24000.0,250200.000000,10.2,NaN,5.0,4.0,2.0,1.0,3.0,0.0,4.0,2.0,1.0,0.0,NaN,0.0,0.0,NaN,NaN,1.0,8,0
2841,DE-987303-01,2.0,81.0,NaN,1.0,1.0,0.0,NaN,0.0,0.0,0.0,18414.087666,0.0,0.000000,0.0,0.0,22043.488477,24000.0,159957.684939,14.0,NaN,3.0,3.0,3.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,NaN,0.0,0.0,NaN,NaN,1.0,8,0
2846,DE-987811-01,1.0,75.0,NaN,1.0,1.0,0.0,NaN,0.0,0.0,0.0,14400.000000,400.0,0.000000,0.0,0.0,22064.807031,42000.0,710707.598509,11.0,NaN,3.0,4.0,2.0,1.0,1.0,0.0,0.0,2.0,1.0,0.0,NaN,0.0,0.0,1.0,NaN,1.0,8,0


In [26]:
cols = ["mstat"]
result_df = process_gv_imputations(2, cols)

assert result_df['mergeid'].nunique() == len(result_df)
assert result_df['mstat'].count() == len(result_df)

result_df["mstat"].count(), result_df["mstat"].isna().sum()

(2628, 0)

In [27]:
wave = 2
module = "dn"
module_file = SRC / f"data/sharew{wave}/sharew{wave}_rel8-0-0_{module}.dta"
dn = pd.read_stata(module_file, convert_categoricals=False)
dn = dn[dn["country"] == 12]

dn["dn014_"].count(), dn["dn014_"].isna().sum(), dn["dn014_"].shape

(1102, 1526, (2628,))

In [28]:
def load_and_rename_wave_data(wave):

    module = "sp"
    module_file = SRC / f"data/sharew{wave}/sharew{wave}_rel8-0-0_{module}.dta"

    data = pd.read_stata(module_file, convert_categoricals=False)
    data.columns = [col[:-2] if col.endswith("sp") else col for col in data.columns]

    return data

In [29]:
data = load_and_rename_wave_data(wave=4)
data[data["sp019d1"].notna()]["sp019d1"]

31       1.0
34       1.0
112      0.0
138      1.0
168      1.0
        ... 
57940    0.0
57957    0.0
57960    0.0
57965    0.0
57973    1.0
Name: sp019d1, Length: 3905, dtype: float64

In [30]:
def filter_nested_dict(original_dict, keys_to_remove):
    return {
        key: [value for value in values if value not in keys_to_remove.get(key, [])]
        if key in keys_to_remove
        else values
        for key, values in original_dict.items()
    }

In [31]:
def process_wave(wave_number, data_modules):
    wave_data = {}

    for module in data_modules.keys():
        print(f"module: {module}")
        module_file = (
            SRC / f"data/sharew{wave_number}/sharew{wave_number}_rel8-0-0_{module}.dta"
        )

        # Read and filter
        if module in ["re", "rp"] and wave_number == 7:
            wave_module = pd.read_stata(module_file, convert_categoricals=False)
            wave_module = wave_module[wave_module["country"] == 12]

            lookup = {
                f"{var[3:]}": f"{var}"
                for var in data_modules[module]
                if var.startswith("sl")
            }
            
        else:
            wave_module = pd.read_stata(module_file, convert_categoricals=False)
            # wave_module = wave_module[wave_module["country"] == "Germany"]
            wave_module = wave_module[wave_module["country"] == 12]

            lookup = {
                "sp009_1sp": "sp009_1",
                "sp009_2sp": "sp009_2",
                "sp009_3sp": "sp009_3",
                "sp019d1sp": "sp019d1",
                "sp019d2sp": "sp019d2",
                "sp019d3sp": "sp019d3",
                "sp019d4sp": "sp019d4",
                "sp019d5sp": "sp019d5",
                "sp019d6sp": "sp019d6",
                "sp019d7sp": "sp019d7",
                "sp019d8sp": "sp019d8",
                "sp019d9sp": "sp019d9",
                "sp019d10sp": "sp019d10",
                "sp019d11sp": "sp019d11",
                "sp019d12sp": "sp019d12",
                "sp019d13sp": "sp019d13",
                "sp019d14sp": "sp019d14",
                "sp019d15sp": "sp019d15",
                "sp019d16sp": "sp019d16",
                "sp019d17sp": "sp019d17",
                "sp019d18sp": "sp019d18",
                "sp019d19sp": "sp019d19",
                "sp019d20sp": "sp019d20",
                "sp019d21sp": "sp019d21",
                # received personal care within household
                "sp021d1sp": "sp021d1",
                "sp021d2sp": "sp021d2",
                "sp021d3sp": "sp021d3",
                "sp021d4sp": "sp021d4",
                "sp021d5sp": "sp021d5",
                "sp021d6sp": "sp021d6",
                "sp021d7sp": "sp021d7",
                "sp021d8sp": "sp021d8",
                "sp021d9sp": "sp021d9",
                "sp021d10sp": "sp021d10",
                "sp021d11sp": "sp021d11",
                "sp021d12sp": "sp021d12",
                "sp021d13sp": "sp021d13",
                "sp021d14sp": "sp021d14",
                "sp021d15sp": "sp021d15",
                "sp021d16sp": "sp021d16",
                "sp021d17sp": "sp021d17",
                "sp021d18sp": "sp021d18",
                "sp021d19sp": "sp021d19",
                "sp021d20sp": "sp021d20",
                "sp021d21sp": "sp021d21",
            }
            # result_dict = {f"sp021d{num}sp": f"sp021d{num}" for num in range(1, 22)}

        # Rename columns using the dictionary
        wave_module.rename(columns=lookup, inplace=True)

        module_vars = ["mergeid"] + data_modules[module]

        # Select columns
        wave_module = wave_module[module_vars]

        wave_data[module] = wave_module

        print(wave_module.shape)

    add_wealth_data = "gv_imputations" in data_modules
    merged_data = wave_data["cv_r"]

    data_modules.pop("cv_r")
    data_modules.pop("gv_imputations", None)

    for module_key in data_modules.keys():
        merged_data = merged_data.merge(
            wave_data[module_key], on="mergeid", how="outer"
        )

    if add_wealth_data:
        merged_data = merged_data.merge(
            wave_data["gv_imputations"], on="mergeid", how="left"
        )

    merged_data["wave"] = wave_number

    return merged_data

In [32]:
all_variables = {
    "cv_r": [
        "int_year",
        "int_month",
        "gender",
        "mobirth",
        "yrbirth",
        "age_int",
        "hhsize",
    ],
    "dn": [
        "dn002_",
        "dn003_",
        "dn019_",  # widowed since when
        "dn010_",
        "dn041_",
        "dn009_",
        "dn014_",
        "dn015_",
        "dn016_",
        "dn026_1",
        "dn026_2",
        "dn033_1",
        "dn033_2",
        "dn027_1",
        "dn027_2",
        "dn028_1",
        "dn028_2",
        "dn030_1",
        "dn030_2",
        "dn034_",  # any siblings
        "dn036_",  # how many brothers alive
        "dn037_",  # how many sisters alive
        "dn127_1",
        "dn127_2",
        "dn032_1",
        "dn032_2",
        "dn012d1",
        "dn012d2",
        "dn012d3",
        "dn012d4",
        "dn012d5",
        "dn012d6",
        "dn012d7",
        "dn012d8",
        "dn012d9",
        "dn012d10",
        "dn012d11",
        "dn012d12",
        "dn012d13",
        "dn012d14",
        "dn012d15",
        "dn012d16",
        "dn012d17",
        "dn012d18",
        "dn012d19",
        "dn012d20",
        # "dn012d95",
        "dn012dno",
        "dn012dot",
    ],
    "ep": [
        "ep005_",
        "ep013_",
        "ep002_",
        "ep071d1",  # income sources, pension payments
        "ep071d2",  # income sources, pension payments
        "ep071d3",  # income sources, pension payments
        "ep071d4",  # income sources, pension payments
        "ep071d5",  # income sources, pension payments
        "ep071d6",  # income sources, pension payments
        "ep071d7",  # income sources, pension payments
        "ep071d8",  # income sources, pension payments
        "ep071d9",  # income sources, pension payments
        "ep071d10",  # income sources, pension payments
        "ep328_",
        "ep329_",
        "ep213_1",
        # "ep213_2",
        # "ep213_3",
        # "ep213_4",
        # "ep213_5",
        # "ep213_6",
        # "ep213_7",
        # "ep213_8",
        # "ep213_9",
        # "ep213_10",
        # "ep213_11",
        # "ep213_12",
        # "ep213_13",
        # "ep213_14",
        # "ep213_15",
        # "ep213_16",
    ],
    "sp": [
        # outside household
        "sp008_",  # given help outside
        "sp009_1",  # to whom given help outside 1
        "sp009_2",  # to whom given help outside 2
        "sp009_3",  # to whom given help outside 3
        "sp010d1_1",  # help given person 1: personal care
        "sp010d1_2",  # help given person 2: personal care
        "sp010d1_3",  # help given person 3: personal care
        # only wave 1 and 2
        # sp012_1, # number of hours practical help
        #
        "sp011_1",  # how often given help to person 1
        "sp011_2",  # how often given help to person 1
        "sp011_3",  # how often given help to person 1
        "sp013_1",  # GiveHelpToOth
        "sp013_2",  # GiveHelpToOth
        # "sp013_3",  # GiveHelpToOth
        # within household
        "sp018_",  # given help within
        "sp019d1",  # provided help with personal care to: spouse/partner
        "sp019d2",  # provided help with personal care to: mother
        "sp019d3",  # provided help with personal care to: father
        "sp019d4",
        "sp019d5",
        "sp019d6",
        "sp019d7",
        "sp019d8",
        "sp019d9",
        "sp019d10",
        "sp019d11",
        # "sp019d12",
        # "sp019d13",
        # "sp019d14",
        # "sp019d15",
        # "sp019d16",
        # "sp019d17",
        # "sp019d18",
        # "sp019d19",
        # "sp019d20",
        # received personal care in household
        "sp020_",  # someone in this household helped you regularly with personal care
        "sp021d1",  # R received help with personal care from: spouse/partner
        "sp021d10",  # child 1
        "sp021d11",  # child 2
        "sp021d12",  # child 3
        "sp021d13",  # child 4
        "sp021d14",  # child 5
        "sp021d15",  # child 6
        "sp021d16",  # child 7
        "sp021d17",  # child 8
        "sp021d18",  # child 9
        "sp021d19",  # child other
        "sp021d20",  # son in law
        "sp021d21",  # daughter in law
    ],
    "gv_isced": ["isced1997_r"],
    # "gv_imputations": [
    #    "hnetw"
    # ],  # household net worth = total gross financial assets + total real assets - total libailities
    "ch": ["ch001_"],
}

In [33]:
keys_to_remove_wave1 = {
    "dn": [
        "dn041_",
        "dn127_1",
        "dn127_2",
        "dn012d15",
        "dn012d16",
        "dn012d17",
        "dn012d18",
        "dn012d19",
        "dn012d20",
    ],
    "ep": [
        "ep013_",
        "ep328_",
        "ep329_",
        "ep213_12",
        "ep213_13",
        "ep213_14",
        "ep213_15",
        "ep213_16",
    ],
}

keys_to_remove_wave2 = {
    "dn": [
        "dn127_1",
        "dn127_2",
        "dn012d15",
        "dn012d16",
        "dn012d17",
        "dn012d18",
        "dn012d19",
        "dn012d20",
    ],
}


keys_to_remove_wave4 = {
    "dn": [
        "dn127_1",
        "dn127_2",
        "dn012d14",
        "dn012d15",
        "dn012d16",
        "dn012d17",
        "dn012d18",
        "dn012d19",
        "dn012d20",
    ],
    "sp": [
        # type of help not answered, assume help includes personal care
        "sp010d1_1",  # help given person 1: personal care
        "sp010d1_2",  # help given person 2: personal care
        "sp010d1_3",  # help given person 3: personal care
        # within household personal care to brother etc.
        # "sp019d8",
        # "sp019d9",
        "sp019d10",
        "sp019d11",
        # provided help with personal care to child 3 - 9
        "sp019d12",
        "sp019d13",
        "sp019d14",
        "sp019d15",
        "sp019d16",
        "sp019d17",
        "sp019d18",
        "sp019d19",
        "sp019d20",
        #
        # received help with personal care from child 3 - 9
        "sp021d10",
        "sp021d11",
        "sp021d12",
        "sp021d13",
        "sp021d14",
        "sp021d15",
        "sp021d16",
        "sp021d17",
        "sp021d18",
        "sp021d19",
        "sp021d20",
        "sp021d21",
    ],
}


keys_to_remove_wave5 = {
    "dn": [
        "dn127_1",  # year of death mother
        "dn127_2",  # year of death father
        "dn012d20",  # further educ category 20
        "dn012dno",  # further educ none
    ],
    # type of help not answered, assume help includes personal care
    "sp": [
        "sp010d1_1",  # help given person 1: personal care
        "sp010d1_2",  # help given person 2: personal care
        "sp010d1_3",  # help given person 3: personal care
    ],
}

keys_to_remove_wave6 = {
    "dn": [
        "dn012dno",
    ],
    "ep": [
        "ep213_14",
        "ep213_15",
        "ep213_16",
        "ep071d1",  # income sources, pension payments
        "ep071d2",  # income sources, pension payments
        "ep071d3",  # income sources, pension payments
        "ep071d4",  # income sources, pension payments
        "ep071d5",  # income sources, pension payments
        "ep071d6",  # income sources, pension payments
        "ep071d7",  # income sources, pension payments
        "ep071d8",  # income sources, pension payments
        "ep071d9",  # income sources, pension payments
        "ep071d10",  # income sources, pension payments
    ],
    # provided help with personal care to child 3 - 9
    # "sp": [
    #    "sp019d12",
    #    "sp019d13",
    #    "sp019d14",
    #    "sp019d15",
    #    "sp019d16",
    #    "sp019d17",
    #    "sp019d18",
    #    "sp019d19",
    # ],
    # received help with personal care from child 3 - 9
    "sp": [
        "sp021d12",
        "sp021d13",
        "sp021d14",
        "sp021d15",
        "sp021d16",
        "sp021d17",
        "sp021d18",
        "sp021d19",
    ],
}

keys_to_remove_wave7 = {
    "dn": [
        "dn012dno",
    ],
    "ep": [
        "ep213_14",
        "ep213_15",
        "ep213_16",
        "ep071d1",  # income sources, pension payments
        "ep071d2",  # income sources, pension payments
        "ep071d3",  # income sources, pension payments
        "ep071d4",  # income sources, pension payments
        "ep071d5",  # income sources, pension payments
        "ep071d6",  # income sources, pension payments
        "ep071d7",  # income sources, pension payments
        "ep071d8",  # income sources, pension payments
        "ep071d9",  # income sources, pension payments
        "ep071d10",  # income sources, pension payments
    ],
    "re": [
        "sl_re018_1",
        "sl_re018_2",
        "sl_re018_3",
        "sl_re018_4",
        "sl_re018_5",
        "sl_re018_6",
        "sl_re018_7",
        "sl_re018_8",
        "sl_re018_9",
        "sl_re018_10",
        "sl_re018_11",
        "sl_re018_12",
        "sl_re018_13",
        "sl_re018_14",
        "sl_re018_15",
        "sl_re018_16",
        "sl_re018_17",
        "sl_re020_1",
        "sl_re020_2",
        "sl_re020_3",
        "sl_re020_4",
        "sl_re020_5",
        "sl_re020_6",
        "sl_re020_7",
        "sl_re020_8",
        "sl_re020_9",
        "sl_re020_10",
        "sl_re020_11",
        "sl_re020_12",
        "sl_re020_13",
        "sl_re020_14",
        "sl_re020_15",
        "sl_re020_16",
        "sl_re020_17",
    ],
    "rp": [
        "sl_rp003_18",
        "sl_rp008_6",
    ],  # year started living with partner, year married
    # provided help with personal care to child 3 - 9
    # "sp": [
    #    "sp019d12",
    #    "sp019d13",
    #    "sp019d14",
    #    "sp019d15",
    #    "sp019d16",
    #    "sp019d17",
    #    "sp019d18",
    #    "sp019d19",
    # ],
    # received help with personal care from child 3 - 9
    "sp": [
        "sp021d12",
        "sp021d13",
        "sp021d14",
        "sp021d15",
        "sp021d16",
        "sp021d17",
        "sp021d18",
        "sp021d19",
    ],
}

keys_to_remove_wave8 = {
    "dn": [
        "dn012dno",
    ],
    "ep": [
        "ep213_14",
        "ep213_15",
        "ep213_16",
        "ep071d1",  # income sources, pension payments
        "ep071d2",  # income sources, pension payments
        "ep071d3",  # income sources, pension payments
        "ep071d4",  # income sources, pension payments
        "ep071d5",  # income sources, pension payments
        "ep071d6",  # income sources, pension payments
        "ep071d7",  # income sources, pension payments
        "ep071d8",  # income sources, pension payments
        "ep071d9",  # income sources, pension payments
        "ep071d10",  # income sources, pension payments
    ],
    # provided help with personal care to child 3 - 9
    # "sp": [
    #    "sp019d12",
    #    "sp019d13",
    #    "sp019d14",
    #    "sp019d15",
    #    "sp019d16",
    #    "sp019d17",
    #    "sp019d18",
    #    "sp019d19",
    # ],
    # received help with personal care from child 3 - 9
    "sp": [
        "sp021d12",
        "sp021d13",
        "sp021d14",
        "sp021d15",
        "sp021d16",
        "sp021d17",
        "sp021d18",
        "sp021d19",
    ],
}

In [34]:
# Retrospective waves
re_vars = (
    [f"sl_re011_{i}" for i in range(1, 21)]
    + [f"sl_re016_{i}" for i in range(1, 21)]
    + [f"sl_re026_{i}" for i in range(1, 21)]
    + [f"sl_re018_{i}" for i in range(1, 17)]
    + [f"sl_re020_{i}" for i in range(1, 17)]
)
rp_vars = (
    ["sl_rp002_", "sl_rp002d_", "sl_rp002e_"]
    + [f"sl_rp003_{i}" for i in range(11, 19)]  # year started living with partner
    + [f"sl_rp004b_{i}" for i in range(1, 6)]  # year started living with married partner
    # year started living with partner
    + [f"sl_rp004c_{i}" for i in [1, 2, 3, 4, 5, 11, 12, 13, 14, 15, 16, 17]]
    + [f"sl_rp008_{i}" for i in range(1, 7)]  # year married
    + [f"sl_rp013_{i}" for i in range(1, 5)]  #  divorced partner
    + [f"sl_rp014_{i}" for i in range(1, 5)]  # year divorced partner
)

rp_vars_wave3 = [
    f"sl_rp011_{i}" for i in [1, 2, 3, 11, 12, 13, 14, 15]
]  # 16 year death of partner
rp_vars_wave7 = [f"sl_rp011_{i}" for i in [1, 2, 3, 4, 11, 12, 13, 14, 15]]  #

# Data modules for wave 3
variables_wave3 = {
    "cv_r": [
        "int_year",
        "int_month",
        "gender",
        "mobirth",
        "yrbirth",
        "age_int",
        "hhsize",
    ],
    "re": re_vars,
    "rp": rp_vars + rp_vars_wave3,
}

# Separate modules for partly retrospective wave 7
variables_wave7 = filter_nested_dict(
    all_variables | {"re": re_vars, "rp": rp_vars + rp_vars_wave7}, keys_to_remove_wave7
)

In [35]:
variables_wave1 = filter_nested_dict(all_variables, keys_to_remove_wave1)
variables_wave2 = filter_nested_dict(all_variables, keys_to_remove_wave2)
variables_wave4 = filter_nested_dict(all_variables, keys_to_remove_wave4)
variables_wave5 = filter_nested_dict(all_variables, keys_to_remove_wave5)
variables_wave6 = filter_nested_dict(all_variables, keys_to_remove_wave6)
variables_wave8 = filter_nested_dict(all_variables, keys_to_remove_wave8)

In [36]:
wave1 = process_wave(wave_number=1, data_modules=variables_wave1)
wave2 = process_wave(wave_number=2, data_modules=variables_wave2)
wave3 = process_wave(wave_number=3, data_modules=variables_wave3)
wave4 = process_wave(wave_number=4, data_modules=variables_wave4)
wave5 = process_wave(wave_number=5, data_modules=variables_wave5)
wave6 = process_wave(wave_number=6, data_modules=variables_wave6)
wave7 = process_wave(wave_number=7, data_modules=variables_wave7)
wave8 = process_wave(wave_number=8, data_modules=variables_wave8)

module: cv_r
(3920, 8)
module: dn
(2995, 40)
module: ep
(2995, 14)
module: sp
(2995, 39)
module: gv_isced
(2995, 2)
module: ch
(2995, 2)
module: cv_r
(3504, 8)
module: dn
(2628, 41)
module: ep
(2628, 17)
module: sp
(2628, 39)
module: gv_isced
(2628, 2)
module: ch
(2628, 2)
module: cv_r
(2501, 8)
module: re
(1918, 93)
module: rp
(1918, 51)
module: cv_r
(2146, 8)
module: dn
(1619, 40)
module: ep
(1619, 17)
module: sp
(1619, 22)
module: gv_isced
(1619, 2)
module: ch
(1619, 2)
module: cv_r
(7674, 8)
module: dn
(5750, 45)
module: ep
(5750, 17)
module: sp
(5750, 36)
module: gv_isced
(5750, 2)
module: ch
(5750, 2)
module: cv_r
(5787, 8)
module: dn
(4411, 48)
module: ep
(4411, 7)
module: sp
(4411, 31)
module: gv_isced
(4411, 2)
module: ch
(4411, 2)
module: cv_r
(4928, 8)
module: dn
(3820, 48)
module: ep
(3820, 7)
module: sp
(3820, 31)
module: gv_isced
(3820, 2)
module: ch
(3820, 2)
module: re
(3820, 61)
module: rp
(3820, 50)
module: cv_r
(4202, 8)
module: dn
(2878, 48)
module: ep
(2878, 7)
mod

In [37]:
# merge in gv imputations by wave
gv_vars = [
    "gender",
    "age",  # Age of respondent (based on interview year)
    "age_p",  # Age of partner (based on interview year)
    "mstat", # Marital status
    "single",
    "couple",
    "partner",
    "nursinghome",
    # "perho",
    "ydip",
    "yind",
    "ypen1",  # Annual old age, early retirement pensions, survivor and war pension
    "ypen2",  # Annual private occupational pensions
    "ypen5",  # Annual payment from social assistance
    "yreg1",  # Other regular payments from private pensions
    "yreg2",  # Other regular payment from private transfer
    "thinc",  # Total household net income - version A
    "thinc2",  # Total household net income - version B
    "hnetw",  # Household net worth (hnfass + hrass)
    "yedu",
    "yedu_p",
    "isced",
    "sphus",  # Self-perceived health – US scale
    "nchild",  # Number of children
    "gali",  # Limitation with activities: PH005
    "chronic",  # Number of chronic diseases: PH006
    "adl",  # Limitations with activities of daily living: PH049_1
    "iadl",  # Limitations with instrumental activities of daily living: PH049_2
    "eurod",  # EURO depression scale: MH002-MH017 (MH031)
    "cjs",  # Current job situation: EP005
    "pwork",  # Did any paid work: EP002
    "empstat",  # Employee or self-employed: EP009; 2- 8
    "rhfo",  # Received help from others (how many): SP002, SP005, SP007
    "ghto",  # Given help to others (how many): SP008, SP011, SP013
    "ghih",  # Given help in the household (how many): SP0181 2 4 5 6 7 (R) 8
    "rhih",  # Received help in the household (how many): SP0201 2 4 5 6 7 (R) 8
    "otrf",  # Owner, tenant or rent free: HO0021 2 4 5 6 7 (R) 8
]

In [38]:
gv_wave1 = process_gv_imputations(wave=1, args=gv_vars)
gv_wave2 = process_gv_imputations(wave=2, args=gv_vars)
gv_wave4 = process_gv_imputations(wave=4, args=gv_vars)
gv_wave5 = process_gv_imputations(wave=5, args=gv_vars)
gv_wave6 = process_gv_imputations(wave=6, args=gv_vars)
gv_wave7 = process_gv_imputations(wave=7, args=gv_vars)
gv_wave8 = process_gv_imputations(wave=8, args=gv_vars)

wave1_merged = wave1.merge(gv_wave1, on='mergeid', how='left')
wave2_merged = wave1.merge(gv_wave2, on='mergeid', how='left')

wave4_merged = wave1.merge(gv_wave4, on='mergeid', how='left')
wave5_merged = wave1.merge(gv_wave5, on='mergeid', how='left')
wave6_merged = wave1.merge(gv_wave6, on='mergeid', how='left')
wave7_merged = wave1.merge(gv_wave7, on='mergeid', how='left')
wave8_merged = wave1.merge(gv_wave8, on='mergeid', how='left')

In [39]:
# List of dataset variable names
dataset_names = ["wave1", "wave2", "wave3", "wave4", "wave5", "wave6", "wave7", "wave8"]

# Create a list to store the counts of duplicated mergeid values for each dataset
duplicated_counts = []

for dataset_name in dataset_names:
    # Get the DataFrame for the current dataset
    df = globals()[dataset_name]  # Access the DataFrame using the variable name
    
    # Check for duplicates in the 'mergeid' column
    duplicates = df['mergeid'].duplicated().sum()
    
    # Append the count to the list
    duplicated_counts.append((dataset_name, duplicates))

# Print the counts for each dataset
for dataset_name, duplicates in duplicated_counts:
    print(f"{dataset_name}: {duplicates} duplicated mergeid values.")

wave1: 0 duplicated mergeid values.
wave2: 0 duplicated mergeid values.
wave3: 0 duplicated mergeid values.
wave4: 0 duplicated mergeid values.
wave5: 0 duplicated mergeid values.
wave6: 0 duplicated mergeid values.
wave7: 0 duplicated mergeid values.
wave8: 0 duplicated mergeid values.


In [40]:
waves_list = [wave1, wave2, wave3, wave4, wave5, wave6, wave7, wave8]
# waves_list = [wave1, wave2, wave3, wave4, wave5, wave6, wave7]

# Drop all nan rows
for i, df in enumerate(waves_list):
    waves_list[i] = df.dropna(how="all", axis=0, inplace=False)
    # waves_list[i] = df.dropna(axis=1, how='all')

In [41]:
def merge_wave_datasets(wave_datasets):
    # Combine the data frames in wave_datasets into one data frame
    #     combined_data = pd.concat(wave_datasets)
    combined_data = pd.concat(wave_datasets, axis=0, ignore_index=True)

    # Filter out rows where the 'int_year' column is not equal to -9
    # combined_data = combined_data[combined_data["int_year"] != -9]
    # combined_data = combined_data[combined_data["int_year"] != "Not applicable"]
    combined_data = combined_data[combined_data["int_year"] != -9]

    # Sort the data frame by 'mergeid' and 'int_year'
    combined_data = combined_data.sort_values(by=["mergeid", "int_year"])

    return combined_data

In [42]:
data = merge_wave_datasets(waves_list)

In [43]:
data.shape

# (26593, 251)

(26593, 256)

In [44]:
gv_wave_list = [gv_wave1, gv_wave2, gv_wave4, gv_wave5, gv_wave6, gv_wave7, gv_wave8]

# Concatenate the DataFrames vertically
stacked_gv_data = pd.concat(gv_wave_list, axis=0, ignore_index=True)

# Sort the DataFrame by 'mergeid' and 'wave'
stacked_gv_data = stacked_gv_data.sort_values(by=["mergeid", "wave"])

# Reset the index after sorting
stacked_gv_data = stacked_gv_data.reset_index(drop=True)

In [45]:
stacked_gv_data.drop('gender', axis=1, inplace=True)

# Merge 'data' and 'stacked_gv_data' on 'mergeid' and 'wave' with a left join
data = data.merge(stacked_gv_data, on=['mergeid', 'wave'], how='left')

In [46]:
data.shape

# (26593, 251)

(26593, 291)

In [47]:
sum(pd.crosstab(data["int_year"], columns='Count')["Count"])

26590

In [48]:
def table(df_col):
    return pd.crosstab(df_col, columns='Count')["Count"]

# 2) Data preparation

## Discrete state variables

###### states which are interpolated
- labor market experience (0, 30)
- years in retirement (0, 6)
- years in intensive care (0, 5)
- father age (70, 90)
- mother age (70, 90)
###### states which are not interpolated
- individuals’ type (1, 2)
- father died last period (0, 1)
- mother died last period (0, 1)
- father alive (0, 1)
- mother alive (0, 1)
- health of father (1, 2, 3)
- health of mother (1, 2, 3)
- existence of siblings (0, 1) --> **change to existence of sister?**
- parents live close by (0, 1)
- married (0, 1)
- education (low, high)

In [49]:
MIN_AGE, MAX_AGE = 55, 68

In [50]:
# only females
dat = data.copy()

# Filter for females
#dat = dat[dat['gender'] == "Female"]
dat = dat[dat['gender'] == 2]

In [51]:
# Age calculation
dat['age'] = dat.apply(lambda row: row['int_year'] - row['yrbirth'] if row['int_month'] >= row['mobirth'] else row['int_year'] - row['yrbirth'] - 1, axis=1)

# Keep only those aged 55 to 68
dat = dat[(dat['age'] >= MIN_AGE) & (dat['age'] <= MAX_AGE)]

In [52]:
dat.shape

(6823, 291)

## Working (part time full time)

**Retirement** Individuals are considered retired if they respond to be retired in the question on their
current job situation. In addition, individuals are considered retired if they respond **not to be working**
and respond to be **receiving old age pension benefits**.

**Working**  Individuals are considered part-time employed if they respond to be working and provide
a number of working hours within the 5th to 50th percentile of the distribution of working hours. This
corresponds to 10 to 32 hours per week. Individuals are considered full-time employed if they work
more than the median of hours in the distribution of working hours (more than 32 hours per week).
In the model we consider the mass-points of the distribution at the 25th percentile (20hours per week)
and 75th percentile of the distribution (40 hours per week) for working women as part- and full- time
work.

**Years in retirement** Individuals give information on the time they have spend in retirement. If the
information is missing and individuals are considered retired we use information given in SHARElife
to construct retrospectively the year in which the last job ended

In [53]:
dat['working'] = np.where(dat['ep005_'] == 2, 1, np.where(dat['ep005_'] < 0, np.nan, 0))

In [54]:
dat.sort_values(by=["mergeid", "int_year"], inplace=True)

dat["first_int_year"] = dat.groupby("mergeid")["int_year"].transform("first")
dat['lagged_int_year'] = dat.groupby('mergeid')['int_year'].shift(1)
dat['lagged_working'] = dat.groupby('mergeid')['working'].shift(1)

In [55]:
# Identify columns that start with "sl_re011"
job_start = [col for col in dat.columns if col.startswith("sl_re011")]

# Iterate through columns and set values < 0 to NA
for job in job_start:
    dat[job] = np.where(dat[job] < 0, np.nan, dat[job])

# Identify columns that start with "sl_re026"
job_end = [col for col in dat.columns if col.startswith("sl_re026")]

# Iterate through columns and set values < 0 to NA, and values == 9997 to int_year
for job in job_end:
    dat[job] = np.where(dat[job] < 0, np.nan, np.where(dat[job] == 9997, dat["int_year"], dat[job]))

In [56]:
dat["sl_re026_1"].describe()

count    1389.000000
mean     1983.509719
std        15.944905
min      1956.000000
25%      1971.000000
50%      1980.000000
75%      1991.000000
max      2017.000000
Name: sl_re026_1, dtype: float64

In [57]:
table(dat["sl_re026_1"])

sl_re026_1
1956.0     1
1957.0     1
1958.0     7
1959.0     6
1960.0    19
          ..
2013.0     6
2014.0    10
2015.0    11
2016.0     7
2017.0    90
Name: Count, Length: 62, dtype: int64

## Most recent job started

In [58]:
conditions = [(dat["sl_re011_1"].notna() & (dat["wave"] == 3))]
values = [1]

# Use numpy.select to create the 'wave_3_response' variable
dat["wave_3_response"] = np.select(conditions, values, default=np.nan)


# Define the conditions and corresponding values for wave 7 response
conditions_wave_7 = [(dat["sl_re011_1"].notna() & (dat["wave"] == 7))]
values_wave_7 = [1]

# Use numpy.select to create the 'wave_7_response' variable
dat["wave_7_response"] = np.select(conditions_wave_7, values_wave_7, default=np.nan)

In [59]:
dat['wave_3_response'] = dat.groupby('mergeid')['wave_3_response'].transform(lambda x: x.ffill().bfill())
dat['wave_7_response'] = dat.groupby('mergeid')['wave_7_response'].transform(lambda x: x.ffill().bfill())

In [60]:
dat["wave_3_response"] = dat["wave_3_response"].fillna(0)
dat["wave_7_response"] = dat["wave_7_response"].fillna(0)

dat["both_wave_3_and_7"] = (
    (dat["wave_3_response"] == 1) & (dat["wave_7_response"] == 1)
).astype(int)

In [61]:
table(dat["both_wave_3_and_7"])

both_wave_3_and_7
0    6823
Name: Count, dtype: int64

In [62]:
prefixes = ["sl_re011_", "sl_re026_"]

# Iterate over the prefixes and apply forward and backward fill
for prefix in prefixes:
    relevant_cols = [col for col in dat.columns if col.startswith(prefix)]
    dat[relevant_cols] = dat.groupby("mergeid")[relevant_cols].transform(
        lambda x: x.ffill().bfill()
    )

In [64]:
# List of columns starting with "sl_re026_" or "sl_re011_"
columns_to_check = [
    col
    for col in dat.columns
    if col.startswith("sl_re026_") or col.startswith("sl_re011_")
]

# Use map with a lambda function to replace both negative and values greater than or equal to 9997 with np.nan
dat[columns_to_check] = dat[columns_to_check].apply(
    lambda x: x.map(lambda val: np.nan if val < 0 else val)
)

In [65]:
# List of suffix numbers (1 to 20)
suffixes = range(1, 21)

# Initialize an empty DataFrame to store the filtered rows
filtered_rows = pd.DataFrame()

# Iterate through the suffixes and filter the rows
for suffix in suffixes:
    column_name = f'sl_re026_{suffix}'
    filtered_rows = pd.concat([filtered_rows, dat[dat[column_name] > 2020]])

# Display the filtered rows
filtered_rows

,mergeid,int_year,int_month,gender,mobirth,yrbirth,age_int,hhsize,dn002_,dn003_,dn019_,dn010_,dn009_,dn014_,dn015_,dn016_,dn026_1,dn026_2,dn033_1,dn033_2,dn027_1,dn027_2,dn028_1,dn028_2,dn030_1,dn030_2,dn034_,dn036_,dn037_,dn032_1,dn032_2,dn012d1,dn012d2,dn012d3,dn012d4,dn012d5,dn012d6,dn012d7,dn012d8,dn012d9,dn012d10,dn012d11,dn012d12,dn012d13,dn012d14,dn012dno,dn012dot,ep005_,ep002_,ep071d1,ep071d2,ep071d3,ep071d4,ep071d5,ep071d6,ep071d7,ep071d8,ep071d9,ep071d10,ep213_1,sp008_,sp009_1,sp009_2,sp009_3,sp010d1_1,sp010d1_2,sp010d1_3,sp011_1,sp011_2,sp011_3,sp013_1,sp013_2,sp018_,sp019d1,sp019d2,sp019d3,sp019d4,sp019d5,sp019d6,sp019d7,sp019d8,sp019d9,sp019d10,sp019d11,sp020_,sp021d1,sp021d10,sp021d11,sp021d12,sp021d13,sp021d14,sp021d15,sp021d16,sp021d17,sp021d18,sp021d19,sp021d20,sp021d21,isced1997_r,ch001_,wave,dn041_,ep013_,ep328_,ep329_,sl_re011_1,sl_re011_2,sl_re011_3,sl_re011_4,sl_re011_5,sl_re011_6,sl_re011_7,sl_re011_8,sl_re011_9,sl_re011_10,sl_re011_11,sl_re011_12,sl_re011_13,sl_re011_14,sl_re011_15,sl_re011_16,sl_re011_17,sl_re011_18,sl_re011_19,sl_re011_20,sl_re016_1,sl_re016_2,sl_re016_3,sl_re016_4,sl_re016_5,sl_re016_6,sl_re016_7,sl_re016_8,sl_re016_9,sl_re016_10,sl_re016_11,sl_re016_12,sl_re016_13,sl_re016_14,sl_re016_15,sl_re016_16,sl_re016_17,sl_re016_18,sl_re016_19,sl_re016_20,sl_re026_1,sl_re026_2,sl_re026_3,sl_re026_4,sl_re026_5,sl_re026_6,sl_re026_7,sl_re026_8,sl_re026_9,sl_re026_10,sl_re026_11,sl_re026_12,sl_re026_13,sl_re026_14,sl_re026_15,sl_re026_16,sl_re026_17,sl_re026_18,sl_re026_19,sl_re026_20,sl_re018_1,sl_re018_2,sl_re018_3,sl_re018_4,sl_re018_5,sl_re018_6,sl_re018_7,sl_re018_8,sl_re018_9,sl_re018_10,sl_re018_11,sl_re018_12,sl_re018_13,sl_re018_14,sl_re018_15,sl_re018_16,sl_re020_1,sl_re020_2,sl_re020_3,sl_re020_4,sl_re020_5,sl_re020_6,sl_re020_7,sl_re020_8,sl_re020_9,sl_re020_10,sl_re020_11,sl_re020_12,sl_re020_13,sl_re020_14,sl_re020_15,sl_re020_16,sl_rp002_,sl_rp002d_,sl_rp002e_,sl_rp003_11,sl_rp003_12,sl_rp003_13,sl_rp003_14,sl_rp003_15,sl_rp003_16,sl_rp003_17,sl_rp003_18,sl_rp004b_1,sl_rp004b_2,sl_rp004b_3,sl_rp004b_4,sl_rp004b_5,sl_rp004c_1,sl_rp004c_2,sl_rp004c_3,sl_rp004c_4,sl_rp004c_5,sl_rp004c_11,sl_rp004c_12,sl_rp004c_13,sl_rp004c_14,sl_rp004c_15,sl_rp004c_16,sl_rp004c_17,sl_rp008_1,sl_rp008_2,sl_rp008_3,sl_rp008_4,sl_rp008_5,sl_rp008_6,sl_rp013_1,sl_rp013_2,sl_rp013_3,sl_rp013_4,sl_rp014_1,sl_rp014_2,sl_rp014_3,sl_rp014_4,sl_rp011_1,sl_rp011_2,sl_rp011_3,sl_rp011_11,sl_rp011_12,sl_rp011_13,sl_rp011_14,sl_rp011_15,dn012d15,dn012d16,dn012d17,dn012d18,dn012d19,dn127_1,dn127_2,dn012d20,sl_rp011_4,age,age_p,mstat,single,couple,partner,nursinghome,ydip,yind,ypen1,ypen2,ypen5,yreg1,yreg2,thinc,thinc2,hnetw,yedu,yedu_p,isced,sphus,nchild,gali,chronic,adl,iadl,eurod,cjs,pwork,empstat,rhfo,ghto,ghih,rhih,otrf,working,first_int_year,lagged_int_year,lagged_working,wave_3_response,wave_7_response,both_wave_3_and_7


import pandas as pd

# Assuming you have already defined the list of suffixes
suffixes = range(1, 17)

# Create an empty list to store DataFrames
selected_dataframes = []

# Loop through each suffix and select rows where sl_re026_{suffix} < sl_re011_{suffix}
for suffix in suffixes:
    condition = dat[f"sl_re026_{suffix}"] < dat[f"sl_re011_{suffix}"]
    selected_suffix_rows = dat[condition]
    
    # Append the selected DataFrame to the list
    selected_dataframes.append(selected_suffix_rows)

# Concatenate the list of DataFrames into one DataFrame
selected_rows = pd.concat(selected_dataframes, ignore_index=True)

selected_rows[columns_to_check].head()

In [66]:
suffixes = range(1, 17)

for suffix in suffixes:
    dat[f"weight_exper_{suffix}"] = np.nan

    job_ended = np.where(
        dat[f"sl_re026_{suffix}"] >= dat["first_int_year"],
        dat["first_int_year"],
        dat[f"sl_re026_{suffix}"],
    )
    #job_ended = dat[f"sl_re026_{suffix}"]

    always_full_time = dat[f"sl_re016_{suffix}"] == 1.0
    dat.loc[always_full_time, f"weight_exper_{suffix}"] = 1.0 * np.abs(
        job_ended - dat[f"sl_re011_{suffix}"]
    )

    always_part_time = dat[f"sl_re016_{suffix}"] == 2.0
    dat.loc[always_part_time, f"weight_exper_{suffix}"] = 0.5 * np.abs(
        job_ended - dat[f"sl_re011_{suffix}"]
    )

    switched_from_full_to_part_time = dat[f"sl_re016_{suffix}"] == 3.0
    dat.loc[switched_from_full_to_part_time, f"weight_exper_{suffix}"] = 1 * np.abs(
        dat[f"sl_re018_{suffix}"] - dat[f"sl_re011_{suffix}"]
    ) + 0.5 * np.abs(job_ended - dat[f"sl_re018_{suffix}"])

    switched_from_part_to_full_time = dat[f"sl_re016_{suffix}"] == 4.0
    dat.loc[switched_from_part_to_full_time, f"weight_exper_{suffix}"] = 0.5 * np.abs(
        dat[f"sl_re018_{suffix}"] - dat[f"sl_re011_{suffix}"]
    ) + 1 * np.abs(job_ended - dat[f"sl_re020_{suffix}"])

In [67]:
# work experience
suffixes = range(1, 17)

# Create a list of column names for 'weight_exper_' columns
weight_columns = [f'weight_exper_{i}' for i in suffixes]

# Calculate work_experience row-wise and store the result in a new column
dat['_retro_work_exp'] = dat[weight_columns].sum(axis=1)

# Group by 'mergeid' and transform to propagate work_experience value
#dat['work_experience'] = dat.groupby('mergeid')['work_experience'].transform('first')

# Calculate the maximum work_experience value within each 'mergeid' group
#max_work_experience = dat.groupby('mergeid')['work_experience'].transform('max')
dat["retro_work_exp"] = dat.groupby('mergeid')['_retro_work_exp'].transform('max')


# Create a copy of the DataFrame to de-fragment it
dat = dat.copy()

In [68]:
dat["retro_work_exp"].sum()
# 109890.5

109890.5

In [69]:
dat["_retro_work_exp"].sum()

# 31790.5

31790.5